# Amazon Dataset Script

### Import Statements

In [1]:
import pandas as pd
import dask.dataframe as dd
import csv
import shutil
from tempfile import NamedTemporaryFile
from tqdm import tqdm

### Root for csv files

In [2]:
root = "H:\\AmazonDataset\\amazon_reviews_us_"

### Create Customer Data csv File

In [20]:
headers = ['customer_id', 'num_reviews', 'num_five_star_ratings', 'num_four_star_ratings',
          'num_three_star_ratings', 'num_two_star_ratings', 'num_one_star_ratings', 'total_helpful_votes', 'vine',
          'num_verified_purchases']

with open(root+'customer_data.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(headers)
csvfile.close()

### Testing Editing Rows and Appending Rows using Dask

In [33]:
ddf = dd.read_csv(root+"customer_data.csv")
# ddf['num_reviews'] = ddf.customer_id.where(ddf.customer_id == 17747349, ddf.num_reviews+1)
cust = 17747349
no_reviews = 1
helpful_votes = 3
no_five_star_ratings = 1
no_four_star_ratings = 0
no_three_star_ratings = 0
no_two_star_ratings = 0
no_one_star_ratings = 0
verified_purchases = 1
vine = 'N'

ls = [cust, no_reviews, no_five_star_ratings, no_four_star_ratings, no_three_star_ratings, no_two_star_ratings, no_one_star_ratings, helpful_votes, vine, verified_purchases]
row = pd.Series(ls, index=ddf.columns)

if cust in ddf.customer_id:
    ddf['num_reviews'] = ddf.num_reviews.where(ddf.customer_id == cust, ddf.num_reviews-no_reviews) + no_reviews
    ddf['num_five_star_ratings'] = ddf.num_five_star_ratings.where(ddf.customer_id == cust, ddf.num_five_star_ratings-no_five_star_ratings) + no_five_star_ratings
    ddf['num_four_star_ratings'] = ddf.num_four_star_ratings.where(ddf.customer_id == cust, ddf.num_four_star_ratings-no_four_star_ratings) + no_four_star_ratings
    ddf['num_three_star_ratings'] = ddf.num_three_star_ratings.where(ddf.customer_id == cust, ddf.num_three_star_ratings-no_three_star_ratings) + no_three_star_ratings
    ddf['num_two_star_ratings'] = ddf.num_two_star_ratings.where(ddf.customer_id == cust, ddf.num_two_star_ratings-no_two_star_ratings) + no_two_star_ratings
    ddf['num_one_star_ratings'] = ddf.num_one_star_ratings.where(ddf.customer_id == cust, ddf.num_one_star_ratings-no_one_star_ratings) + no_one_star_ratings
    ddf['total_helpful_votes'] = ddf.total_helpful_votes.where(ddf.customer_id == cust, ddf.total_helpful_votes-helpful_votes) + helpful_votes
    ddf['num_verified_purchases'] = ddf.num_verified_purchases.where(ddf.customer_id == cust, ddf.num_verified_purchases-verified_purchases) + verified_purchases
else:
    ddf = ddf.append(row)
   #ddf['num_reviews'] = ddf.num_reviews.where(ddf.customer_id == 17747349, ddf.num_reviews-1) + 1
ddf.compute().to_csv(root+"customer_data_0.csv", index=False)
#ddf.to_csv(root+"customer_data-*.csv", index=False)

### Function For Writing Customer Data Into CSV

In [61]:
def write_into_csv(tsv_data):
    
    nddf = dd.from_pandas(tsv_data, chunksize=128)
    #nddf['num_reviews'] = nddf['num_reviews'].astype(int) 
    ddf = dd.read_csv(root+"customer_data.csv")
    print(nddf)
    print(ddf)

    for i in range(len(nddf)):
        customer_id = tsv_data.values[i][0]
        #print(f"CustomerID: {customer_id}")
        star_rating = tsv_data.values[i][2]
        #print(f"StarRating: {star_rating}")
        helpful_votes = tsv_data.values[i][3]
        #print(f"HelpfulVotes: {helpful_votes}")
        vine = tsv_data.values[i][5]
        num_reviews = 1
        #print(f"NumReviews: {num_reviews}")

        if tsv_data.values[i][6] == 'Y':
            verified_purchases = 1
        else:
            verified_purchases = 0

        if star_rating == 1.0:
            num_five_star_ratings = 0
            num_four_star_ratings = 0
            num_three_star_ratings = 0
            num_two_star_ratings = 0
            num_one_star_ratings = 1
        elif star_rating == 2.0:
            num_five_star_ratings = 0
            num_four_star_ratings = 0
            num_three_star_ratings = 0
            num_two_star_ratings = 1
            num_one_star_ratings = 0
        elif star_rating == 3.0:
            num_five_star_ratings = 0
            num_four_star_ratings = 0
            num_three_star_ratings = 1
            num_two_star_ratings = 0
            num_one_star_ratings = 0
        elif star_rating == 4.0:
            num_five_star_ratings = 0
            num_four_star_ratings = 1
            num_three_star_ratings = 0
            num_two_star_ratings = 0
            num_one_star_ratings = 0
        elif star_rating == 5.0:
            num_five_star_ratings = 1
            num_four_star_ratings = 0
            num_three_star_ratings = 0
            num_two_star_ratings = 0
            num_one_star_ratings = 0

        newRow = [customer_id, num_reviews, num_five_star_ratings, num_four_star_ratings, num_three_star_ratings, num_two_star_ratings,
              num_one_star_ratings, helpful_votes, vine, verified_purchases]
        newRow = pd.Series(ls, index=ddf.columns)
        
        if customer_id in ddf.customer_id:
            print('eddit row')
            print(type(customer_id), type(num_reviews), type(ddf.customer_id), type(ddf.num_reviews), type(ddf['num_reviews']))
            #print(ddf.num_reviews.where(ddf.customer_id == int(customer_id), ddf.num_reviews))
            print(ddf.head())
            ddf['num_reviews'] = ddf.num_reviews.where(ddf.customer_id == int(customer_id), ddf.num_reviews-int(num_reviews)) + int(num_reviews)
            ddf['num_five_star_ratings'] = ddf.num_five_star_ratings.where(ddf.customer_id == customer_id, ddf.num_five_star_ratings-int(num_five_star_ratings)) + int(num_five_star_ratings)
            ddf['num_four_star_ratings'] = ddf.num_four_star_ratings.where(ddf.customer_id == customer_id, ddf.num_four_star_ratings-int(num_four_star_ratings)) + int(num_four_star_ratings)
            ddf['num_three_star_ratings'] = ddf.num_three_star_ratings.where(ddf.customer_id == customer_id, ddf.num_three_star_ratings-int(num_three_star_ratings)) + int(num_three_star_ratings)
            ddf['num_two_star_ratings'] = ddf.num_two_star_ratings.where(ddf.customer_id == customer_id, ddf.num_two_star_ratings-int(num_two_star_ratings)) + int(num_two_star_ratings)
            ddf['num_one_star_ratings'] = ddf.num_one_star_ratings.where(ddf.customer_id == customer_id, ddf.num_one_star_ratings-int(num_one_star_ratings)) + int(num_one_star_ratings)
            ddf['total_helpful_votes'] = ddf.total_helpful_votes.where(ddf.customer_id == customer_id, ddf.total_helpful_votes-int(helpful_votes)) + int(helpful_votes)
            ddf['num_verified_purchases'] = ddf.num_verified_purchases.where(ddf.customer_id == customer_id, ddf.num_verified_purchases-int(verified_purchases)) + int(verified_purchases)
        else:
            print('appening row')
            ddf = ddf.append(newRow)

    ddf.compute().to_csv(root+"customer_data_0.csv", index=False)
        #df = pd.read_csv(root+"customer_data.csv")

#         if customer_id in df.values:
            
#             filename = root + "customer_data.csv"
#             tempfile = NamedTemporaryFile(mode='w', delete=False)
            
#             fields = ['customer_id', 'num_reviews', 'num_five_star_ratings', 'num_four_star_ratings',
#           'num_three_star_ratings', 'num_two_star_ratings', 'num_one_star_ratings', 'total_helpful_votes', 'vine',
#           'num_verified_purchases']
            
#             with open(filename, 'r', newline='') as csvfile, tempfile:
#                 reader = csv.DictReader(csvfile, fieldnames = fields)
#                 writer = csv.DictWriter(tempfile, fieldnames = fields)
#                 for row in reader:
#                     if row['customer_id'] == str(customer_id):
#                         #print('updating customer', row['customer_id'])
#                         row['num_reviews'] = str(int(row['num_reviews']) + int(num_reviews))
#                         row['num_five_star_ratings'] = str(int(row['num_five_star_ratings']) + int(num_five_star_ratings))
#                         row['num_four_star_ratings'] = str(int(row['num_four_star_ratings']) + int(num_four_star_ratings))
#                         row['num_three_star_ratings'] = str(int(row['num_three_star_ratings']) + int(num_three_star_ratings))
#                         row['num_two_star_ratings'] = str(int(row['num_two_star_ratings']) + int(num_two_star_ratings))
#                         row['num_one_star_ratings'] = str(int(row['num_one_star_ratings']) + int(num_one_star_ratings))
#                         row['total_helpful_votes'] = str(int(float(row['total_helpful_votes'])) + int(float(helpful_votes)))
#                         row['vine'] = vine
#                         row['num_verified_purchases'] = str(int(row['num_verified_purchases']) + int(verified_purchases))
                        
#                     row = {'customer_id': row['customer_id'], 'num_reviews': row['num_reviews'], 'num_five_star_ratings': row['num_five_star_ratings'], 
#                           'num_four_star_ratings': row['num_four_star_ratings'], 'num_three_star_ratings': row['num_three_star_ratings'], 'num_two_star_ratings': row['num_two_star_ratings'],
#                           'num_one_star_ratings': row['num_one_star_ratings'], 'total_helpful_votes': row['total_helpful_votes'], 'vine': row['vine'],
#                           'num_verified_purchases': row['num_verified_purchases']}
#                     writer.writerow(row)
                    
#             shutil.move(tempfile.name, filename)
#         else:
#             with open(root+'customer_data.csv', 'a', newline='') as csvfile:
#                 csvwriter = csv.writer(csvfile)
#                 csvwriter.writerow(newRow)
#             csvfile.close()


### Read Apparel_v1_00.tsv

In [18]:
fields = ['customer_id', 'review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase']
batch_num = 0
batch_size = 128

#for j in tqdm(range(len(pd.read_csv(root+"Apparel_v1_00.tsv", sep='\t', error_bad_lines=False, usecols=fields))//batch_size)):
    #Apparel_data = pd.read_csv(root+"Apparel_v1_00.tsv", sep='\t', error_bad_lines=False, usecols=fields, skiprows=range(1, batch_num*batch_size), nrows=batch_size)

Apparel_data = pd.read_csv(root+"Apparel_v1_00.tsv", sep='\t', error_bad_lines=False, usecols=fields)
write_into_csv(Apparel_data)    
    #write_into_csv(Apparel_data)
    #batch_num = batch_num + 1

  0%|          | 39/5881951 [03:34<8970:05:49,  5.49s/it]


ValueError: invalid literal for int() with base 10: '0.0'

### Read Digital_Software_v1_00.tsv

In [45]:
fields = ['customer_id', 'review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase']
batch_num = 0
batch_size = 128

#for j in tqdm(range(len(pd.read_csv(root+"Apparel_v1_00.tsv", sep='\t', error_bad_lines=False, usecols=fields))//batch_size)):
    #Apparel_data = pd.read_csv(root+"Apparel_v1_00.tsv", sep='\t', error_bad_lines=False, usecols=fields, skiprows=range(1, batch_num*batch_size), nrows=batch_size)

Apparel_data = pd.read_csv(root+"Digital_Software_v1_00.tsv", sep='\t', error_bad_lines=False, usecols=fields)
write_into_csv(Apparel_data)    
    #write_into_csv(Apparel_data)
    #batch_num = batch_num + 1

eddit row
<class 'int'> <class 'int'> <class 'dask.dataframe.core.Series'> <class 'dask.dataframe.core.Series'>
Dask Series Structure:
npartitions=1
    int64
      ...
Name: num_reviews, dtype: int64
Dask Name: where, 5 tasks
eddit row
<class 'int'> <class 'int'> <class 'dask.dataframe.core.Series'> <class 'dask.dataframe.core.Series'>
Dask Series Structure:
npartitions=1
    int64
      ...
Name: num_reviews, dtype: int64
Dask Name: where, 61 tasks
eddit row
<class 'int'> <class 'int'> <class 'dask.dataframe.core.Series'> <class 'dask.dataframe.core.Series'>
Dask Series Structure:
npartitions=1
    int64
      ...
Name: num_reviews, dtype: int64
Dask Name: where, 117 tasks
eddit row
<class 'int'> <class 'int'> <class 'dask.dataframe.core.Series'> <class 'dask.dataframe.core.Series'>
Dask Series Structure:
npartitions=1
    int64
      ...
Name: num_reviews, dtype: int64
Dask Name: where, 173 tasks
eddit row
<class 'int'> <class 'int'> <class 'dask.dataframe.core.Series'> <class 'dask

KeyboardInterrupt: 

### Read Gift_Card_v1_00.tsv

In [63]:
fields = ['customer_id', 'review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase']
batch_num = 0
batch_size = 16

for j in tqdm(range(len(pd.read_csv(root+"Gift_Card_v1_00.tsv", sep='\t', error_bad_lines=False, usecols=fields))//batch_size)):
    Gift_Card_data = pd.read_csv(root+"Gift_Card_v1_00.tsv", sep='\t', error_bad_lines=False, usecols=fields, skiprows=range(1, batch_num*batch_size), nrows=batch_size)

#Apparel_data = pd.read_csv(root+"Digital_Software_v1_00.tsv", sep='\t', error_bad_lines=False, usecols=fields)
    write_into_csv(Gift_Card_data)
    batch_num = batch_num + 1


  0%|          | 0/9269 [00:00<?, ?it/s]

Dask DataFrame Structure:
              customer_id review_id star_rating helpful_votes total_votes    vine verified_purchase
npartitions=1                                                                                      
0                   int64    object       int64         int64       int64  object            object
15                    ...       ...         ...           ...         ...     ...               ...
Dask Name: from_pandas, 1 tasks
Dask DataFrame Structure:
              customer_id num_reviews num_five_star_ratings num_four_star_ratings num_three_star_ratings num_two_star_ratings num_one_star_ratings total_helpful_votes    vine num_verified_purchases
npartitions=1                                                                                                                                                                                        
                    int64       int64                 int64                 int64                  int64                

  0%|          | 1/9269 [00:02<6:15:51,  2.43s/it]

Dask DataFrame Structure:
              customer_id review_id star_rating helpful_votes total_votes    vine verified_purchase
npartitions=1                                                                                      
0                   int64    object       int64         int64       int64  object            object
15                    ...       ...         ...           ...         ...     ...               ...
Dask Name: from_pandas, 1 tasks
Dask DataFrame Structure:
              customer_id num_reviews num_five_star_ratings num_four_star_ratings num_three_star_ratings num_two_star_ratings num_one_star_ratings total_helpful_votes    vine num_verified_purchases
npartitions=1                                                                                                                                                                                        
                    int64       int64                 int64                 int64                  int64                

  0%|          | 2/9269 [00:04<6:14:04,  2.42s/it]

Dask DataFrame Structure:
              customer_id review_id star_rating helpful_votes total_votes    vine verified_purchase
npartitions=1                                                                                      
0                   int64    object       int64         int64       int64  object            object
15                    ...       ...         ...           ...         ...     ...               ...
Dask Name: from_pandas, 1 tasks
Dask DataFrame Structure:
              customer_id num_reviews num_five_star_ratings num_four_star_ratings num_three_star_ratings num_two_star_ratings num_one_star_ratings total_helpful_votes    vine num_verified_purchases
npartitions=1                                                                                                                                                                                        
                    int64       int64                 int64                 int64                  int64                

  0%|          | 2/9269 [00:05<7:09:04,  2.78s/it]

  customer_id num_reviews num_five_star_ratings num_four_star_ratings  \
0    17747349           1                     0                     1   
1    10956619           1                     0                     0   
2    13132245           1                     0                     0   
3    35717248           1                     1                     0   
4    17710652           1                     0                     1   

  num_three_star_ratings num_two_star_ratings num_one_star_ratings  \
0                      0                    0                    0   
1                      1                    0                    0   
2                      0                    0                    1   
3                      0                    0                    0   
4                      0                    0                    0   

  total_helpful_votes vine num_verified_purchases  
0                   0    N                      1  
1                   0    N          

ValueError: Metadata inference failed in `sub`.

Original error is below:
------------------------
TypeError("unsupported operand type(s) for -: 'str' and 'int'")

Traceback:
---------
  File "C:\Users\brand\anaconda3\lib\site-packages\dask\dataframe\utils.py", line 175, in raise_on_meta_error
    yield
  File "C:\Users\brand\anaconda3\lib\site-packages\dask\dataframe\core.py", line 5169, in elemwise
    meta = partial_by_order(*parts, function=op, other=other)
  File "C:\Users\brand\anaconda3\lib\site-packages\dask\utils.py", line 1107, in partial_by_order
    return function(*args2, **kwargs)
  File "C:\Users\brand\anaconda3\lib\site-packages\pandas\core\ops\common.py", line 65, in new_method
    return method(self, other)
  File "C:\Users\brand\anaconda3\lib\site-packages\pandas\core\arraylike.py", line 97, in __sub__
    return self._arith_method(other, operator.sub)
  File "C:\Users\brand\anaconda3\lib\site-packages\pandas\core\series.py", line 4998, in _arith_method
    result = 